<a href="https://colab.research.google.com/github/ShobyG/inventory/blob/master/pantry_image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications import VGG16
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras import preprocessing,callbacks

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [ ]:
from tensorflow.keras.models import Model, load_model

In [ ]:
training_set = preprocessing.image_dataset_from_directory("/content/drive/MyDrive/pantry_images/train",
                                                              validation_split=0.2,
                                                              subset="training",
                                                              label_mode="categorical",
                                                              seed=0,
                                                              image_size=(100,100))

Found 87 files belonging to 4 classes.
Using 70 files for training.


In [ ]:
print("Classes:", training_set.class_names)

Classes: ['apples', 'banana', 'bread', 'eggs']


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
x = base_model.output # output layer of the base model

x = GlobalAveragePooling2D()(x)
# a fully-connected layer
x = Dense(1024, activation='relu')(x)
output_layer = Dense(4, activation='softmax')(x)

In [ ]:
m = Model(inputs=base_model.input, outputs=output_layer)

In [ ]:
# train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
callback = callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
m.compile(loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
epochs = 500

In [ ]:
print("Training.")
history = m.fit(training_set, batch_size=5, epochs=epochs,verbose=1, callbacks = [callback])
print(history.history["accuracy"])

Training.
Epoch 1/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 1.0000 - loss: 4.2813e-07 
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 1.0000 - loss: 3.8490e-07 
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 1.0000 - loss: 3.8476e-07 
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 1.0000 - loss: 4.0044e-07 
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 3.3385e-07
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 4.0301e-07
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 3.9952e-07
Epoch 8/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 3.6344e-07
Epoch 9/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 3.0454e-07
Epoch 10/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 3.4017e-07
Epoch 11/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 3.1907e-07
Epoch 12/500
3/3

In [ ]:
m.save("/content/drive/MyDrive/pantry_images/vgg_fine_tuned.h5")

In [ ]:
test_set = preprocessing.image_dataset_from_directory("/content/drive/MyDrive/pantry_images/test",
                                                          validation_split=0.2,
                                                          subset="validation",
                                                          label_mode="categorical",
                                                          seed=0,
                                                          image_size=(100, 100))

Found 8 files belonging to 4 classes.
Using 1 files for validation.


In [ ]:
score = m.evaluate(test_set, verbose=0)

In [ ]:
print('Test accuracy:', score[1])

Test accuracy: 1.0


In [ ]:
img = preprocessing.image.load_img("/content/drive/MyDrive/pantry_images/test/apples/Screenshot 2024-08-01 223550.jpg", target_size=(100, 100))
img_arr = preprocessing.image.img_to_array(img)
img_cl = img_arr.reshape(1, 100, 100, 3)

score = m.predict(img_cl)
predicted_class = np.argmax(score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [ ]:
predicted_class = np.argmax(score)
print(predicted_class)

0


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
m = load_model("/content/drive/MyDrive/pantry_images/vgg_fine_tuned.h5")

In [ ]:
img = preprocessing.image.load_img("/content/drive/MyDrive/pantry_images/cam_pic/egg_2.jpg", target_size=(100, 100))
img_arr = preprocessing.image.img_to_array(img)
img_cl = img_arr.reshape(1, 100, 100, 3)

score = m.predict(img_cl)
predicted_class = np.argmax(score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [ ]:
print(predicted_class)

3


In [ ]:
img_path = "/content/drive/MyDrive/pantry_images/cam_pic/bread.jpg"
img = Image.open(img_path).resize((100, 100))
# Convert image to array and normalize
img_array = np.array(img) / 255.0
img_array = img_array.astype(np.float32)  # Convert to float32
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

In [ ]:
score = m.predict(img_array)
predicted_class = np.argmax(score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [ ]:
print(predicted_class)

3


In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('/content/drive/MyDrive/pantry_images/vgg_fine_tuned.h5')
print("Original model input shape:", model.input_shape)

Original model input shape: (None, None, None, 3)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

# Assuming your model architecture is accessible
input_layer = Input(shape=(100, 100, 3))
output_layer = model(input_layer)
new_model = Model(inputs=input_layer, outputs=output_layer)
print("Rebuilt model input shape:", new_model.input_shape)

Rebuilt model input shape: (None, 100, 100, 3)


In [ ]:
new_model.save('/content/drive/MyDrive/pantry_images/new_model.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model = converter.convert()

# Save the converted model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpywdiybbi'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 100, 3), dtype=tf.float32, name='keras_tensor_446')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  140498850638736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140496977791984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140496977802896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140496977800080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140496977790576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140496977827040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140496977826512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140496977828096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140496977825280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140496977823520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1404969778

In [ ]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("TFLite model input shape:", input_details[0]['shape'])

TFLite model input shape: [  1 100 100   3]


In [ ]:
from PIL import Image
import numpy as np


In [ ]:
def preprocess_image(image_path):
  img = preprocessing.image.load_img(image_path, target_size=(100, 100))
  img_arr = preprocessing.image.img_to_array(img)
  img_cl = img_arr.reshape(1, 100, 100, 3)
  return img_cl

In [ ]:
image_path = '/content/drive/MyDrive/pantry_images/cam_pic/egg_2.jpg'
input_data = preprocess_image(image_path)

In [ ]:
interpreter.set_tensor(input_details[0]['index'], input_data)

In [ ]:
interpreter.invoke()

In [ ]:
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[1.5840390e-01 3.6235861e-03 1.8322943e-08 8.3797252e-01]]


In [ ]:
predicted_class = np.argmax(output_data[0])

In [ ]:
print(predicted_class)

3


In [ ]:
 !pip3 install tensorflowjs

In [ ]:
import tensorflow as tf

# Load your Keras model
model = tf.keras.models.load_model('/content/drive/MyDrive/pantry_images/new_model.h5')

# Save the model in TensorFlow SavedModel format
model.export('/content/drive/MyDrive/pantry_images/night_stupidity')


Saved artifact at '/content/drive/MyDrive/pantry_images/night_stupidity'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  135812082561120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135812082564816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135812082765664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135812082551968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135812082554960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135812082550208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135812082555664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135812082768656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135812082763024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135812082771648: TensorSpec(shape=(), dtype=t

In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model /content/drive/MyDrive/pantry_images/night_stupidity /content/drive/MyDrive/pantry_images/night_js


2024-08-05 07:55:30.460380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 07:55:30.495446: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 07:55:30.495528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 07:55:30.517009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 07:55:32.028175: W tensorflow/compiler/tf